In [67]:
import numpy as np
from scipy.io.wavfile import read as wavread
from scipy.fft import fft
from scipy.signal.windows import hann
from scipy import signal
import matplotlib.pyplot as plt
import math
import os
import librosa
from IPython.display import Audio

## From earlier Homework assignments

In [2]:
def  block_audio(x,blockSize,hopSize,fs):
    # allocate memory
    numBlocks = math.ceil(x.size / hopSize)
    xb = np.zeros([numBlocks, blockSize])
    # compute time stamps
    t = (np.arange(0, numBlocks) * hopSize) / fs

    x = np.concatenate((x, np.zeros(blockSize)),axis=0)

    for n in range(0, numBlocks):
        i_start = n * hopSize
        i_stop = np.min([x.size - 1, i_start + blockSize - 1])

        xb[n][np.arange(0,blockSize)] = x[np.arange(i_start, i_stop + 1)]
    return (xb,t)

def ToolReadAudio(cAudioFilePath):
    [samplerate, x] = wavread(cAudioFilePath)

    if x.dtype == 'float32':
        audio = x
    else:
        # change range to [-1,1)
        if x.dtype == 'uint8':
            nbits = 8
        elif x.dtype == 'int16':
            nbits = 16
        elif x.dtype == 'int32':
            nbits = 32

        audio = x / float(2**(nbits - 1))

    # special case of unsigned format
    if x.dtype == 'uint8':
        audio = audio - 1.

    return (samplerate, audio)

In [3]:
def compute_spectrogram(xb, fs):
    window = signal.hann(xb.shape[1])
    windowed_signal = xb*window
    X_jw = fft(windowed_signal)
    X = abs(X_jw)
    f = np.arange(X.shape[1])*fs/X.shape[1]
    bins = f.size // 2 + 1 
    return X[:,:bins].T, f[:bins]

In [5]:
os.listdir()

['Strokes4.ipynb',
 'Feature Extraction.ipynb',
 'cross_validation.pkl',
 'Strokes2.ipynb',
 'letters',
 '.DS_Store',
 'requirements.txt',
 'baseline.py',
 'realtime.py',
 'filtered_nn.pkl',
 'dtw_matcher.py',
 'Extract_Letters.ipynb',
 'Strokes.ipynb',
 'spans.py',
 '__pycache__',
 'Strokes3.ipynb',
 'feature_extraction.py',
 'Strokes_Sk.ipynb',
 'filtered_nn2.pkl',
 'README.md',
 'strokes1.py',
 'Baseline.ipynb',
 '.gitignore',
 'Dataset Collection Instructions.pdf',
 'evaluate.py',
 '.ipynb_checkpoints',
 '.git',
 'mfcc.py',
 'Test.ipynb',
 'hmm.py',
 'span_tweaks.py']

In [7]:
fs, x = ToolReadAudio("letters/1/A/PZ/0.wav")
Audio(x, rate=fs)

In [47]:
def extract_spectral_centroid(xb, fs):
    X, f = compute_spectrogram(xb, fs)
    spec_centroid = np.sum(((X.T)*f),axis=1)/np.sum(X.T, axis=1)
    return spec_centroid

In [64]:
# Not working yet, using librosa for now
# def extract_spectral_flux(xb, fs):
#     X, f = compute_spectrogram(xb, fs)
#     X = X.T
#     diff = np.concatenate((np.zeros((1,X.shape[1])), X[:-1]), axis=0)
#     print(X.shape)
#     spectral_flux = np.sqrt(np.sum(np.square(X-diff), axis=1))/(X.shape[1]+1)
#     return spectral_flux

def extract_spectral_flux(x, fs)
    return librosa.onset.onset_strength(x,sr=fs)

## From MFCC.py

In [76]:
from evaluate import load_dataset
import librosa
import numpy as np
from dtw import dtw
import evaluate

letters, fs = load_dataset()
letters = letters[:, :4, :]

a = letters[0, 0, 0].astype(float) / np.iinfo(np.int32).max

import matplotlib.pyplot as plt
# librosa.feature.mfcc(a).shape
# plt.specgram(a)
# plt.imshow(librosa.feature.mfcc(a), origin='lower')

flux = None
def preprocessor(letters, fs):
    global flux
    fluxes = np.empty(letters.shape, dtype=object)
    for i, letter in enumerate(letters):
        letter = letter.astype(float) / np.iinfo(np.int32).max
        fluxes[i] = librosa.onset.onset_strength(letter,sr=fs)
    return np.arange(len(fluxes)).reshape(-1, 1)

def dtw_dist(a, b):
    a = mfccs[int(a[0])]
    b = mfccs[int(b[0])]
    return dtw(a, b, dist_method="euclidean").normalizedDistance

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

cls = KNeighborsClassifier(1, metric=dtw_dist, algorithm='brute', n_jobs=8)

X = preprocessor(letters.reshape(-1), fs)
y = np.indices(letters.shape)[1].reshape(-1)
subjects = np.indices(letters.shape)[0].reshape(-1)
indices = np.indices(letters.shape).reshape((letters.ndim, -1)).T

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
mask = subjects == 0
X_train, X_test, y_train, y_test, index_train, index_test = train_test_split(
    X[mask], y[mask], indices[mask],
    test_size=0.25,
    stratify=y[mask]
)

cls.fit(X_train, y_train)
y_pred = cls.predict(X_test)
(y_pred == y_test).mean()
ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred, display_labels=evaluate.alphabet[:4],
    include_values=False
)

import evaluate
evaluate.run(preprocessor, cls, subset=np.arange(4))


TypeError: 'NoneType' object is not subscriptable